# 0. Library

In [1]:
import os, time, datetime
os.environ["TOKENIZERS_PARALLELISM"] = "false"
from tqdm import tqdm
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:100% !important;}</style>"))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM


# 1. Config

In [2]:
class Config:
    data_name = "data/dialogue.csv"
    bos_token = '<s>'
    eos_token = '</s>'
    usr_token = '<usr>'
    pad_token = '<pad>'
    sys_token = '<sys>'
    unk_token = '<unk>'
    mask_token = '<mask>'
    max_length = 2 ** 8
    batch_size = 2 ** 3
    epochs = 2 ** 2
    pretrained_model_name = "skt/kogpt2-base-v2"
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    learning_rate = 3e-5
    model_name = 'model.pt'


# 2. Data

In [3]:
raw_data = pd.read_csv(Config.data_name)

In [4]:
N = len(raw_data)
data = raw_data.sample(N)
data

,conversation
178983,<usr> 결혼식 날에 신부 입장할때 너무 뭉클했어<sys> 나는 결혼식하는 것만 ...
193936,<usr> 댄스 말고 배워보고 싶은거 있어?<sys> 댄스 말고는 악기 배우고 싶어...
118970,<usr> 나 도착했어 <sys> 오 일찍왔네?기장님이 좀 밟으셧나봐ㅋㅋ 짐 찾았어...
141438,<usr> 오늘도 축구 예선전이 있다고 하네요<sys> 어디서 중계 하려나요?<us...
151905,<usr> 니 자동차 별칭은 따로 안 정했어?<sys> 그냥 한 번씩 흰둥이라고 불...
...,...
100692,<usr> 혜경자지말고 식순 정해친구도 연습할시간 주구대본도 짜줘야하니 <sys> ...
160230,<usr> 너 현금 쓰면 현금 영수증 끊어?<sys> 웅 당연히 끊지 ㅋㅋ아빠 껄로...
162693,<usr> 아직도 장애인들이 대중교통을 이용할 때 많이 불편한가 보더라고<sys> ...
67764,<usr> 점심시간 <sys> 응응 맛난 거 먹장! <usr> 콩국수요 <sys> ...


In [5]:
def valid_size(train_X):
    length = len(train_X)
    return round(0.14 * (1 + length / 10 ** 4) ** (10 ** 4 / length) - 0.13, 2)

In [6]:
data_train, data_valid = train_test_split(data, test_size=valid_size(data), shuffle=True)
data_train.reset_index(inplace=True)
data_valid.reset_index(inplace=True)

In [7]:
data_train

,index,conversation
0,191862,<usr> 일남 할아버지 근데 죽잖아<sys> 죽는 장면이 안나와..<usr> 아하...
1,40611,<usr> 오 피우는 비타민이란거있대연기도나고ㅋㅋ <sys> 헐?!진짜?산다 <us...
2,38757,<usr> 저녁으로 먹는거 토마토주스맛나는거 내 안무야겠따 하루종일까스 개쩖 엄마눙...
3,136104,<usr> 우리는 다 배가 부르단다 ㅋㅋ뭘 먹었는지 모르게 헛배불러 ㅋㅋ인자 내 씻...
4,117244,<usr> 아글고 아까 저 괴상하게 생긴 굿즈 <sys> 벌써웃겨 <usr> 집에 ...
...,...,...
188635,50657,<usr> 어플로 보면 할인되는거 아이폰은 안나와ㅠㅠ난 몇개월할인받아서 끝나면 해지...
188636,191844,<usr> 양평 쪽에도 좋은 집이 많더라<sys> 서울이랑 거리도 가까워서 그런가?...
188637,175005,<usr> 부모님께 선물은 어버이날만 사드린다는 고정관념이 생겼어요<sys> 전 생...
188638,7962,<usr> 나 이민규야 달고싶은데진짜 삐질거같아서참을게 <sys> 정미현야 ㅠㅠ시스...


In [8]:
data_valid

,index,conversation
0,163158,<usr> 안녕형 나 야식 추천 좀!<sys> 야식은 족발 아니겠어? ㅋㅋ족발 어때...
1,145555,<usr> 나 네일을 받아볼까?<sys> 네일 받아 보는거 나쁘지 않지<usr> 네...
2,77339,<usr> 핸드폰 테마를바꾸고 싶은뎅추천해종 <sys> 음.. 지금 너가 쓰는건파스...
3,167021,<usr> 너 예전에 별에서 온 그대 드라마 봤어?<sys> 네 저 그거 봤죠 ㅋㅋ...
4,101071,<usr> 아프지않게 주사놓는게어렵지않나 <sys> 레이저할 때 아프지않게주사놪 다...
...,...,...
5830,38533,<usr> 아진짜 <sys> 건강하게 <usr> 건강상할정도로만 <sys> 다먹는거...
5831,182716,<usr> 아이돌 굿즈 종류도 정말 많더라<sys> 어떤게 인기 있어?<usr> 기...
5832,59997,<usr> 개피곤하구만..역학부터채점한다 ㅎ ㅎ <sys> ㅎ..하고말해주라 <us...
5833,56714,<usr> 한자외우는데 너무 외롭다..이건 외로운 싸움이야90개외우고 50개틀린듯ㅠ...


# 3. Tokenizer

In [9]:
tokenizer = AutoTokenizer.from_pretrained(Config.pretrained_model_name,
            bos_token=Config.bos_token, eos_token=Config.eos_token,
            unk_token=Config.unk_token, pad_token=Config.pad_token,
            mask_token=Config.mask_token, model_max_length=Config.max_length)

In [10]:
for i in range(10):
    print(tokenizer.convert_ids_to_tokens(i), end=' ')

<s> </s> <usr> <pad> <sys> <unk> <mask> <d> </d> <unused0> 

# 4. Dataloader

In [11]:
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, Config):
        self.data = data
        self.tokenizer = tokenizer
        self.bos_token = Config.bos_token
        self.eos_token = Config.eos_token
        self.usr_token = Config.usr_token
        self.pad_token = Config.pad_token
        self.sys_token = Config.sys_token
        self.max_length = Config.max_length
        
    def __len__(self):
        return len(self.data)
        
    def __getitem__(self, idx):
        sentence = self.data['conversation'][idx]
        # input_id
        input_id = self.tokenizer.encode(self.bos_token + sentence + self.eos_token)
        # token_type_id
        token_type = []
        loop = True
        for token_id in input_id:
            token = self.tokenizer.convert_ids_to_tokens(token_id)
            
            if token == self.usr_token: loop=True
            elif token == self.sys_token: loop=False
                
            if loop:
                token_type.append(self.usr_token)
            else:
                token_type.append(self.sys_token)
        token_type_id = self.tokenizer.convert_tokens_to_ids(token_type)
        # label
        start_idx = len(input_id) - \
            list(reversed(input_id)).index(self.tokenizer.convert_tokens_to_ids(self.sys_token))
        label = [-100] * start_idx + input_id[start_idx: ]
        # padding
        input_id, token_type_id, label = self.make_padding(input_id, token_type_id, label)
        
        return input_id, token_type_id, label

    def make_padding(self, input_id, token_type_id, label):
        left_length = self.max_length - len(input_id)
        input_id += [self.tokenizer.pad_token_id] * left_length
        token_type_id += [self.tokenizer.pad_token_id] * left_length
        label += [-100] * left_length
        
        return input_id, token_type_id, label

In [12]:
train_set = CustomDataset(data_train, tokenizer, Config)

In [13]:
input_id, token_type_id, label = train_set[0]
print("input_id", input_id, sep='\n')
print("token_type_id", token_type_id, sep='\n')
print("label", label, sep='\n')

input_id
[0, 2, 9043, 7062, 32857, 9252, 7220, 9393, 8162, 7965, 4, 24736, 36713, 9183, 16621, 9705, 2, 9050, 8702, 9705, 36550, 9339, 11117, 12964, 406, 4, 9032, 8774, 9760, 9089, 20759, 12312, 7532, 36759, 9390, 12752, 7489, 25571, 2, 20713, 9105, 8146, 8015, 7182, 9407, 7501, 8148, 6855, 15247, 406, 4, 11403, 11525, 44235, 6872, 6853, 7172, 9658, 14902, 13348, 9138, 14990, 11902, 6853, 8263, 9705, 8420, 7501, 20430, 6857, 406, 9252, 7220, 11432, 8220, 8000, 14668, 9022, 6855, 9622, 8137, 6853, 7991, 2, 11403, 9043, 7623, 9021, 23598, 6853, 7991, 406, 10278, 4, 19896, 8018, 9252, 7220, 23971, 9042, 9313, 14909, 9025, 9080, 6853, 8041, 12896, 9705, 8006, 8456, 14927, 9337, 9025, 11218, 9705, 406, 2, 9063, 24107, 9847, 8420, 8015, 7182, 12118, 7545, 7379, 8020, 739, 605, 605, 4, 739, 605, 605, 35704, 9572, 50624, 10056, 7055, 7788, 29144, 7788, 9181, 12306, 6853, 8263, 47318, 10030, 9847, 8420, 9350, 8353, 8137, 6857, 406, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 

In [14]:
def collate_fn(batch):
    input_ids = [items[0] for items in batch]
    token_type_ids = [items[1] for items in batch]
    labels = [items[2] for items in batch]
    
    return torch.LongTensor(input_ids), torch.LongTensor(token_type_ids), \
            torch.LongTensor(labels)

In [15]:
train_dataloader = DataLoader(train_set, batch_size=Config.batch_size, num_workers=2,
                              shuffle=True, collate_fn=collate_fn)

In [16]:
valid_set = CustomDataset(data_valid, tokenizer, Config)
valid_dataloader = DataLoader(valid_set, batch_size=Config.batch_size, num_workers=2,
                            shuffle=False, collate_fn=collate_fn)

In [17]:
input_id, token_type_id, label = valid_set[0]
print("input_id", input_id, sep='\n')
print("token_type_id", token_type_id, sep='\n')
print("label", label, sep='\n')

input_id
[0, 2, 25906, 8745, 9063, 9893, 7889, 13815, 11732, 376, 4, 9893, 11187, 13429, 7601, 9320, 6872, 8006, 406, 739, 605, 605, 8214, 7601, 9105, 7312, 2, 9050, 9077, 7471, 739, 605, 605, 8214, 7601, 22238, 19104, 739, 605, 605, 4, 739, 605, 605, 9050, 13429, 7601, 31187, 6958, 19202, 6972, 12218, 8346, 8214, 7601, 739, 7318, 6960, 7182, 2, 739, 605, 605, 9050, 12218, 8346, 8214, 7601, 9036, 39749, 9183, 33146, 7662, 9668, 11355, 8155, 8139, 406, 4, 10278, 7756, 32951, 10510, 8159, 12011, 9511, 47317, 12011, 8705, 9122, 9239, 7220, 406, 2, 739, 605, 605, 9105, 9063, 10253, 12011, 9328, 6947, 6853, 11355, 18479, 9277, 9293, 11629, 406, 4, 9346, 8148, 8191, 11848, 10464, 9383, 739, 605, 605, 41732, 33146, 7661, 2, 9050, 9022, 6853, 9351, 9293, 15713, 11629, 739, 605, 605, 9114, 8052, 7970, 7415, 8244, 7788, 9065, 7495, 9173, 376, 4, 739, 605, 605, 9054, 16539, 7098, 6853, 6958, 17088, 11355, 49067, 12790, 33146, 7661, 739, 605, 605, 2, 739, 605, 605, 40057, 406, 13429, 7601, 18339, 

# 5. Model

In [18]:
model = AutoModelForCausalLM.from_pretrained(Config.pretrained_model_name).to(Config.device)

# 6. Train

In [19]:
class Train:
    def __init__(self, model, tokenizer, Config):
        self.model = model
        self.tokenizer = tokenizer
        self.optimizer = torch.optim.AdamW(model.parameters(), lr=Config.learning_rate)
#         self.scheduler = torch.optim.lr_scheduler.ExponentialLR(self.optimizer, gamma=0.9)
        self.bos_token = Config.bos_token
        self.eos_token = Config.eos_token
        self.usr_token = Config.usr_token
        self.sys_token = Config.sys_token
        self.device = Config.device
        self.train_losses = []
        self.valid_losses = []

    def train(self, epochs, train_dataloader, valid_dataloader=None, save=False):
        for epoch in range(epochs):
            print(f"Epoch: {epoch + 1} / {epochs}")
            self.model.train()
            losses = []
            start_time = time.time()

            for i, batch in enumerate(train_dataloader):
                input_ids, token_type_ids, labels = batch        
                input_ids, token_type_ids, labels = \
                    input_ids.to(self.device), token_type_ids.to(self.device), labels.to(self.device)
                
                outputs = self.model(
                    input_ids = input_ids,
                    token_type_ids = token_type_ids,
                    labels = labels
                )
                
                loss = outputs.loss
                
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

                losses.append(loss.item())
                train_loss = np.mean(losses)
                print(self.status(i + 1, len(train_dataloader), time.time() - start_time,
                                  train_loss), end='\r')
                
#             self.scheduler.step()
            self.train_losses.append(train_loss)
            
            if valid_dataloader:
                valid_loss = self.validation(valid_dataloader)
                print(self.status(i + 1, len(train_dataloader), time.time() - start_time,
                                train_loss) + f" #valid_loss: {valid_loss:.6f}\n", end='\r')
                self.valid_losses.append(valid_loss)
            
            if save:
                time_zone = datetime.timezone(datetime.timedelta(hours=9))
                now = datetime.datetime.now(time_zone)
                PATH = now.strftime(f'models/%m%d_%H%M_ep{epoch + 1}.pt')
                torch.save(self.model.state_dict(), PATH)

    def validation(self, valid_dataloader):
        self.model.eval()
        losses = []
        
        with torch.no_grad():
            for i, batch in enumerate(valid_dataloader):
                input_ids, token_type_ids, labels = batch
                input_ids, token_type_ids, labels = \
                    input_ids.to(self.device), token_type_ids.to(self.device), labels.to(self.device)
                
                outputs = self.model(
                    input_ids = input_ids,
                    token_type_ids = token_type_ids,
                    labels = labels
                )
                
                loss = outputs.loss
                losses.append(loss.item())
            
            valid_loss = np.mean(losses)
        
        return valid_loss
    
    def status(self, step, steps, past_time, train_loss):
        return f"#step: {step} / {steps} #past: {int(past_time)}s #left: {int(steps / step * time - time)}s #train_loss: {train_loss:.6f}"

    def save(self, PATH=None):
        if not PATH:
            time_zone = datetime.timezone(datetime.timedelta(hours=9))
            now = datetime.datetime.now(time_zone)
            PATH = now.strftime(f'models/%m%d_%H%M_ep{epochs}.pt')
            
        torch.save(self.model.state_dict(), PATH)
        print("model saved.")


In [20]:
chathuman = Train(model, tokenizer, Config)

In [ ]:
chathuman.train(Config.epochs, train_dataloader, valid_dataloader, True)
# chathuman.save(f'models/{Config.model_name}')

Epoch: 1 / 4
#step: 23580 / 23580 #past: 13389s #left: 0s #train_loss: 4.269815 #valid_loss: 4.106904
Epoch: 2 / 4
#step: 23580 / 23580 #past: 13400s #left: 0s #train_loss: 3.899102 #valid_loss: 4.036246
Epoch: 3 / 4


In [ ]:
plt.plot(chathuman.train_losses)
plt.plot(chathuman.valid_losses)
plt.show()